In [10]:
# @ 크롤링시 이상한 이미지, 정보들 들어가는 문제~ 
# 나온거 페이지 안에서 다 비교후 가장 참여 많은것 을 크롤링 하자.   


title = '돈'

from urllib import parse
import re
import requests
from bs4 import BeautifulSoup
import ssl
import urllib.request as req

def get_soup(url):
    ssl._create_default_https_context = ssl._create_unverified_context
    res = req.urlopen(url)
    soup = BeautifulSoup(res, 'lxml')
    return soup

# 한글을 query 로 변환
title_encoded = parse.quote(title)

# 검색 url접속후 detail 페이지로 가기위한 url을 먼저 얻겠다.
url = "https://movie.naver.com/movie/search/result.nhn?query=" + title_encoded + "&section=all&ie=utf8"
soup = get_soup(url)

# 만약 아무것도 검색 결과 없다면 None 입력되고 아래 if문 못들어감.
# detail_url_pre = soup.select_one('ul.search_list_1 > li > dl > dt > a')
detail_url_pre = soup.select('ul.search_list_1 > li > dl > dd.point > em.cuser_cnt')

if detail_url_pre: # 비면(검색결과 x) 아예 아래로 안가겠슴. 
    
  # 모델 객체 저장할 값들 초기화
        pic_url =""
        rating = 0
        genre = ""
        director =""
        age =""
        story =""

        # 이하는 검색 결과 중 가장 많이본 영화 테그 착기 위함.
        max_num = 0
        max_num_rated_movies_line1 = detail_url_pre[0]  # 일단 맨앞 line저장
        for line1 in detail_url_pre:
            #         print(line1) # 여기기준  이웃 , 부모 테그 타고가서 링크 얻자.
            text_pre = line1.get_text()
            number_of_rated_people = re.findall("(\d+)", text_pre)[0]
            #         print(number_of_rated_people)
            if max_num <= int(number_of_rated_people):
                max_num = int(number_of_rated_people)
                max_num_rated_movies_line1 = line1

        parent_tag = max_num_rated_movies_line1.parent
        dt_tag_as_previous_tag = parent_tag.find_previous_sibling("dt")
        a_tag = dt_tag_as_previous_tag.select_one('a')

        # detail로 가는 url
        detail_url = 'https://movie.naver.com' + a_tag.get('href')

        soup = get_soup(detail_url)

        # 포스터 이미지
        pic_url_pre = soup.select_one('div.poster > a > img')
        if pic_url_pre:
            pic_url = pic_url_pre.get('src')
            print(f'pic_url: {pic_url}')

        # 포토가 있는 경우에는 포토 의 맨앞에거 가져다 쓰고싶다.
        pic_viewer_url_pre = soup.select_one('div.viewer_img >  img')
        if pic_viewer_url_pre:
            pic_url = pic_viewer_url_pre.get('src')
            print(f'pic_url: {pic_url}')

        rating_pre = soup.select_one('div.mv_info > div.main_score > div.score.score_left > div.star_score > a')
        if rating_pre:
            rating = rating_pre.get_text()
            print(f'rating: {rating}')


        genre_pre_pre = soup.select_one('div.mv_info > dl.info_spec > dt.step1')
        if genre_pre_pre:
            genre_pre = genre_pre_pre.find_next_siblings('dd')[0].select('p > span > a')[0] # 여기서 인덱스 에러 날수도 있다...
            if genre_pre:
                genre =  genre_pre.get_text()
                print(f'genre: {genre}')


        director_pre_pre = soup.select_one('div.mv_info > dl.info_spec > dt.step2')
        if director_pre_pre:
            director_pre = director_pre_pre.find_next_siblings('dd')[0]
            if director_pre:
                director = director_pre.get_text()
                print(f'director: {director}')


        age_pre_pre = soup.select_one('div.mv_info > dl.info_spec > dt.step4')
        if age_pre_pre:
            age_pre = age_pre_pre.find_next_siblings('dd')[0].select_one('a')
            if age_pre:
                age = age_pre.get_text()
                print(f'age: {age}')

        story_pre = soup.select_one('div.story_area > p.con_tx')
        
#         print(story_pre)
        
        if story_pre:
            story = story_pre.get_text()

            # 제작노트 같이 크롤링 된다면 잘라서 저장 하겠다.


            print(f'story: {story}')

#         # # 썸네일이 이미 없는 경우에만 업데이트를 하자 ------> 일단 안지우겠슴.
#         # if movie.thumbnail_url == "":
#         #     movie.thumbnail_url = pic_url

#         # 썸네일 네이버 검색 이미지 -> 뷰어이미지 첫번째거로 통일 시키겠다.
#         # 무조건 업데이트 하겠슴.(검색 안되는데 섬네일 있는경우는 여기 아예 안들언옴) ---> 12.31/ get_or_create시 계속 중복 생성해서 걍 무조건 여기서 업데이트 하겠슴.
#         movie.thumbnail_url = pic_url                                         # 검색 안되는것 추후 구글검색 첫이미지로 업데이트 시키겠다.

#         # 수지윙의 세계의 경우 네이버 영화 에서 검색은 되지만 섬네일 없다.
#         # 이런경우에는 구글에서 이미지 검색해서 썸네일 업데이트 해주자.
#         if not pic_url:
#             upadete_google_image(movie,movie.title)

#         movie.rating = float(rating)
#         movie.genre = genre
#         movie.director = director
#         movie.age = age
#         movie.story = story
#         movie.save()

    # 네이버 영화에서 검색결과 없다면.


pic_url: https://movie-phinf.pstatic.net/20190306_280/1551849045570X4iac_JPEG/movie_image.jpg?type=m77_110_2
pic_url: https://movie-phinf.pstatic.net/20190208_197/1549586594982x70KK_JPEG/movie_image.jpg?type=m427_320_2
rating: 7.58
genre: 범죄
director: 박누리
age: 15세 관람가
story: 오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현(류준열).  빽도 줄도 없는, 수수료 O원의 그는 곧 해고 직전의 처지로 몰린다. 위기의 순간, 베일에 싸인 신화적인 작전 설계자 번호표(유지태)를 만나게 되고,  막대한 이익을 챙길 수 있는 거래 참여를 제안 받는다.  위험한 제안을 받아들인 후 순식간에 큰 돈을 벌게 되는 일현.  승승장구하는 일현 앞에 번호표의 뒤를 쫓던 금융감독원의 사냥개 한지철(조우진)이 나타나  그를 조여 오기 시작하는데…
